In [1]:
import matplotlib.pyplot as plt
import numpy as np
import timeit

from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord, GCRS, ICRS, EarthLocation, solar_system_ephemeris, get_body_barycentric

from kbmod.reprojection_utils import correct_parallax

Compute the intersection of a sphere centered at the barycenter (0, 0, 0) with radius D and a ray starting at a viewing point on Earth (ex, ey, ez) and pointing in direction (vx, vy, vz).

In [2]:
def correct_parallax2(coord, obstime, point_on_earth, heliocentric_distance):
    """Calculate the parallax corrected postions for a given object at a given time and distance from Earth.

    Attributes
    ----------
    coord : `astropy.coordinate.SkyCoord`
        The coordinate to be corrected for.
    obstime : `astropy.time.Time` or `string`
        The observation time.
    point_on_earth : `astropy.coordinate.EarthLocation`
        The location on Earth of the observation.
    heliocentric_distance : `float`
        The guess distance to the object from the Sun.

    Returns
    ----------
    An `astropy.coordinate.SkyCoord` containing the ra and dec of the point in ICRS, and the best fit geocentric distance (float).

    References
    ----------
    .. [1] `Jupyter Notebook <https://github.com/DinoBektesevic/region_search_example/blob/main/02_accounting_parallax.ipynb>`_
    """
    # Compute the Earth location relative to the barycenter.
    times = Time(obstime, format="mjd")
    
    # Compute the Earth's location in to cartesian space centered the barycenter.
    # This is an approximate position. Is it good enough?
    earth_pos_cart = get_body_barycentric("earth", times)
    ex = earth_pos_cart.x.value
    ey = earth_pos_cart.y.value
    ez = earth_pos_cart.z.value

    # Compute the unit vector of the pointing.
    loc = (
        point_on_earth.x.to(u.m).value,
        point_on_earth.y.to(u.m).value,
        point_on_earth.z.to(u.m).value,
    ) * u.m
    los_earth_obj = coord.transform_to(GCRS(obstime=obstime, obsgeoloc=loc))
    pointings_cart = los_earth_obj.cartesian
    vx = pointings_cart.x.value
    vy = pointings_cart.y.value
    vz = pointings_cart.z.value

    # Solve the quadratic equation for the ray leaving the earth and intersecting
    # a sphere around the sun (0, 0, 0) with radius = heliocentric_distance
    a = vx * vx + vy * vy + vz * vz
    b = 2 * vx * ex + 2 * vy * ey + + 2 * vz * ez
    c = ex * ex + ey * ey + ez * ez - heliocentric_distance * heliocentric_distance
    disc = b * b - 4 * a * c

    if (disc < 0):
        return None, -1.0
    
    # Since the ray will be starting from within the sphere (we assume the 
    # heliocentric_distance is at least 1 AU), one of the solutions should be positive
    # and the other negative. We only use the positive one.
    dist = (-b + np.sqrt(disc))/(2 * a)

    answer = SkyCoord(
        ra=coord.ra,
        dec=coord.dec,
        distance=dist * u.AU,
        obstime=obstime,
        obsgeoloc=loc,
        frame="gcrs",
    ).transform_to(ICRS())

    return answer, dist

In [3]:
icrs_ra1 = 88.74513571
icrs_dec1 = 23.43426475
icrs_time1 = Time("2023-03-20T16:00:00", format="isot", scale="utc")

icrs_ra2 = 91.24261107
icrs_dec2 = 23.43437467
icrs_time2 = Time("2023-09-24T04:00:00", format="isot", scale="utc")

sc1 = SkyCoord(ra=icrs_ra1, dec=icrs_dec1, unit="deg")
sc2 = SkyCoord(ra=icrs_ra2, dec=icrs_dec2, unit="deg")

with solar_system_ephemeris.set("de432s"):
    eq_loc = EarthLocation.of_site("ctio")

In [4]:
# Compute the results.
corrected_coord1, geo_dist1 = correct_parallax(
    coord=sc1,
    obstime=icrs_time1,
    point_on_earth=eq_loc,
    heliocentric_distance=50.0,
)
print("Current Approach")
print(f"Distance = {geo_dist1}")
print(corrected_coord1)

corrected_coord2, geo_dist2 = correct_parallax2(
    coord=sc1,
    obstime=icrs_time1,
    point_on_earth=eq_loc,
    heliocentric_distance=50.0,
)
print("Approximate Approach")
print(f"Distance = {geo_dist2}")
print(corrected_coord2)

Current Approach
Distance = 50.00135417530472
<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, AU)
    (90., 23.43952556, 49.99999999)>
Approximate Approach
Distance = 50.00131315305139
<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, AU)
    (89.99996581, 23.4395254, 49.99995841)>


In [5]:
%%timeit
corrected_coord1, geo_dist1 = correct_parallax(
    coord=sc1,
    obstime=icrs_time1,
    point_on_earth=eq_loc,
    heliocentric_distance=50.0,
)

94.1 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
corrected_coord2, geo_dist2 = correct_parallax2(
    coord=sc1,
    obstime=icrs_time1,
    point_on_earth=eq_loc,
    heliocentric_distance=50.0,
)

3 ms ± 29.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
